# Wrapping a subset of a very large library

In this example, we aim at presenting the methodology to wrap only a subset of a library.
For the purpose of illustration and to demonstrate the usage of **AutoWIG** for very complex libraries, we considered the wrapping of the **Clang** library, a complete *C*/*C++* compiler.
**Clang** is a great tool, but its stable *Python* interface (i.e. **libclang**) is lacking some useful features that are needed by **AutoWIG**.
In particular, class template specializations are not available in the abstract syntax tree.
Fortunately, most of the classes that would be needed during the traversal of the *C++* abstract syntax tree are not template specializations.
We therefore proposed to bootstrap the **Clang** *Python* bindings with a first version of **AutoWIG** that uses only **libclang**.
The new **Clang** *Python* interface, produced by **AutoWIG**, is **PyClangLite**.
**PyClangLite** is able to wrap class template specializations.
As for **libclang**, this interface is proposed only for a subset of the **Clang** library sufficient enough for proposing the new `pyclanglite` `parser`.

First, ensure that the **LLVM**/**Clang** technologies are installed on your computer and if there were built with `RTTI`.

In [1]:
import subprocess
if not subprocess.check_output(["llvm-config", "--version"]).strip() == '3.8.1':
    raise Exception
subprocess.check_call(['clang++', '--version'])
if not subprocess.check_output(["llvm-config", "--has-rtti"]).strip() == 'YES':
    raise Exception

Note that these techonologies must be installed using the current system prefix

In [2]:
import sys
from path import path
prefix = path(sys.prefix)
if not prefix == subprocess.check_output(["llvm-config", "--prefix"]).strip():
    raise Exception

Then, clone the **PyClangLite**  repository into the **PyClangLite** directory.

In [3]:
%%bash
git clone https://github.com/StatisKit/PyClangLite.git

This repository already has wrappers, we therefore need to remove them.

In [4]:
srcdir = 'PyClangLite'/'src'/'py'
for wrapper in srcdir.walkfiles('*.cpp'):
    wrapper.unlink()
for wrapper in srcdir.walkfiles('*.h'):
    wrapper.unlink()
wrapper = srcdir/'clanglite'/'_clanglite.py'
if wrapper.exists():
    wrapper.unlink()

In addition to the **Clang** libraries, the **ClangLite** library is needed in order to have access to some functionalities.
The `tool.h` header of this **ClangLite** library includes all necessary **Clang** headers.

In [5]:
%%bash
scons cpp -C PyClangLite

We import **AutoWIG** and create an empty Abstract Semantic Graph (ASG).

In [6]:
import autowig
asg = autowig.AbstractSemanticGraph()

We then parse the header with relevant compilation flags.

In [7]:
autowig.parser.plugin = 'libclang'
asg = autowig.parser(asg, [prefix/'include'/'clanglite'/'tool.h'],
               flags = ['-x', 'c++', '-std=c++11',
                        '-D__STDC_LIMIT_MACROS', '-D__STDC_CONSTANT_MACROS',
                        '-I' + str((prefix/'include').abspath())],
               libpath = prefix/'lib'/'libclang.so',
               bootstrap = False,
               silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation could be suitable.
Nevertheless, we need to force some *C++* components to be wrapped or not.
Moreover, **libclang** does not allow to investigate if enumerations are scoped or not.
We therefore implements a new `controller`.

In [10]:
def clanglite_controller(asg):
    
    for node in asg['::boost::python'].classes(nested = True):
        node.is_copyable = True
        
    for node in asg.classes():
        node.boost_python_export = False
    for node in asg.enumerations():
        node.boost_python_export = False
    for node in asg.enumerators():
        if node.parent.boost_python_export:
            node.boost_python_export = False
    for node in asg.typedefs():
        node.boost_python_export = False
            
    from autowig.default_controller import refactoring
    asg = refactoring(asg)
    for fct in asg['::clanglite'].functions():
        if not fct.localname == 'build_ast_from_code_with_args':
            fct.parent = fct.parameters[0].qualified_type.desugared_type.unqualified_type
            
    for node in asg.functions(free = True):
        node.boost_python_export = False
    for node in asg.variables(free = True):
        node.boost_python_export = False
    
    asg['class ::boost::python::api::object'].boost_python_export = True
    asg['class ::boost::python::list'].boost_python_export = True 
    asg['class ::boost::python::str'].boost_python_export = True 

    subset = []
    classes = [asg['class ::clang::QualType'],
               asg['class ::clang::Type'],
               asg['class ::clang::Decl']]
    asg._nodes['class ::clang::QualType']['_is_abstract'] = False
    asg['class ::clang::QualType'].is_copyable = True
    subset += classes
    for cls in classes:
        subset += cls.subclasses(recursive=True)
    subset.append(asg['class ::llvm::StringRef'])
    asg._nodes['class ::llvm::StringRef']['_is_abstract'] = False
    asg['class ::llvm::StringRef'].is_copyable = True
    subset.append(asg['class ::clang::ASTUnit'])
    subset.append(asg['class ::clang::ASTContext'])
    subset.append(asg['class ::clang::SourceManager'])
    for mtd in asg['class ::clang::ASTContext'].methods(pattern='.*getSourceManager.*'):
        if mtd.return_type.globalname == 'class ::clang::SourceManager &':
                mtd.boost_python_export = True
                break
    subset.append(asg['class ::clang::FileID'])
    asg._nodes['class ::clang::FileID']['_is_abstract'] = False
    asg['class ::clang::FileID'].is_copyable = True
    subset.append(asg['class ::clang::SourceLocation'])
    asg._nodes['class ::clang::SourceLocation']['_is_abstract'] = False
    asg['class ::clang::SourceLocation'].is_copyable = True
    subset.append(asg['class ::clang::CXXBaseSpecifier'])
    subset.append(asg['class ::clang::DeclContext'])
    subset.append(asg['class ::clang::TemplateArgument'])
    asg._nodes['class ::clang::TemplateArgument']['_is_abstract'] = False
    asg['class ::clang::TemplateArgument'].is_copyable = True
    subset.append(asg['class ::clang::TemplateArgumentList'])
    subset.append(asg['enum ::clang::Type::TypeClass'])
    subset.append(asg['enum ::clang::AccessSpecifier'])
    subset.append(asg['enum ::clang::LinkageSpecDecl::LanguageIDs'])
    subset.append(asg['enum ::clang::BuiltinType::Kind'])
    subset.append(asg['enum ::clang::TemplateArgument::ArgKind'])
    subset.append(asg['enum ::clang::Decl::Kind'])
    subset.extend(asg['::boost::python'].classes(nested = True))
    subset.extend(asg['::boost::python'].enumerations(nested = True))
    subset.extend(asg.nodes('::clanglite::build_ast_from_code_with_args'))

    for node in subset:
        node.boost_python_export = True

    if autowig.parser.plugin == 'libclang':
        for node in (asg.functions(pattern='.*(llvm|clang).*_(begin|end)')
                     + asg.functions(pattern='.*(llvm|clang).*getNameAsString')
                     + asg.nodes('::clang::NamedDecl::getQualifiedNameAsString')
                     + asg.nodes('::clang::ObjCProtocolDecl::collectInheritedProtocolProperties')
                     + asg.nodes('::clang::ASTUnit::LoadFromASTFile')
                     + asg.nodes('::clang::ASTUnit::getCachedCompletionTypes')
                     + asg.nodes('::clang::ASTUnit::getBufferForFile')
                     + asg.nodes('::clang::CXXRecordDecl::getCaptureFields')
                     + asg.nodes('::clang::ASTContext::SectionInfos')
                     + asg.nodes('::clang::ASTContext::getAllocator')
                     + asg.nodes('::clang::ASTContext::getObjCEncoding.*')
                     + asg.nodes('::clang::ASTContext::getAllocator')
                     + asg.nodes('::clang::QualType::getAsString')
                     + asg.nodes('::clang::SourceLocation::printToString')
                     + asg['class ::llvm::StringRef'].methods()):
            node.boost_python_export = False
            
    import sys
    from path import path
    for header in (path(sys.prefix)/'include'/'clang').walkfiles('*.h'):
        asg[header.abspath()].is_external_dependency = False
    
    return asg

This `controller` is then dynamically registered and used on the ASG.

In [11]:
autowig.controller['clanglite'] = clanglite_controller
autowig.controller.plugin = 'clanglite'
asg = autowig.controller(asg)

In order to wrap a subset of the **Clang** library, the user need to implements a `generator` that passes particular nodes to the `boost_python` `generator`.

In [12]:
autowig.generator.plugin = 'boost_python_pattern'
wrappers = autowig.generator(asg,
                  module = srcdir/'_clanglite.cpp',
                  decorator = srcdir/'clanglite'/'_clanglite.py',
                  closure = False)

The wrappers are only generated in-memory.
We therefore need to write them on the disk to complete the process.

In [13]:
wrappers.write()

In [ ]:
%%bash
pygmentize PyClangLite/src/py/_clanglite.cpp